# Image processing pipeline

In [ ]:
from tqdm.notebook import tqdm

import os
import os.path

import pandas as pd
import numpy as np

In [ ]:
# extracts Opera run (measurement) name from the full path
get_run_name = lambda x: x.rstrip("/").split("/")[-1]
# helper functions
contains_pattern = lambda string,patterns: sum(np.array([[p in string] for p in patterns]).flatten())
move_columns_to_front = lambda df, cols: df[cols + [column for column in df.columns if column not in cols]]

## Input

In [ ]:
# will be also the name of the main analysis folder with all processed files
analysis_name = "2023-06-16_HAP1_cancer_41_clones_large_pooled_screen"

In [ ]:
# paths to raw measurements exported from the Opera - after running flatfield correction script from Harmony computer
# space before measurement number needs to be changed to underscore
Opera_runs = [
    "/research/lab_kubicek/jreinis/Opera_images/2023-06-15_HAP1_cancer_41_clones_large_pooled_screen_repeat_suspicousplate/AR_20230614_HAP1_cancer_41clones_pooled_cpdplate_2313010_plate1__2023-06-14T18_00_48-Measurement_1",
    "/research/lab_kubicek/jreinis/Opera_images/2023-06-15_HAP1_cancer_41_clones_large_pooled_screen_repeat_suspicousplate/AR_20230614_HAP1_cancer_41clones_pooled_cpdplate_2313010_plate1__2023-06-15T03_26_51-Measurement_2",
    "/research/lab_kubicek/jreinis/Opera_images/2023-06-15_HAP1_cancer_41_clones_large_pooled_screen_repeat_suspicousplate/AR_20230614_HAP1_cancer_41clones_pooled_cpdplate_2313010_plate2__2023-06-14T21_28_13-Measurement_1",
    "/research/lab_kubicek/jreinis/Opera_images/2023-06-15_HAP1_cancer_41_clones_large_pooled_screen_repeat_suspicousplate/AR_20230614_HAP1_cancer_41clones_pooled_cpdplate_2313010_plate2__2023-06-15T06_54_56-Measurement_2",
]

In [ ]:
# required for hit calling
platemap_path = "/research/lab_kubicek/jreinis/Opera_images/2023-06-15_HAP1_cancer_41_clones_large_pooled_screen_repeat_suspicousplate/JR_20230616_Pool41_Experiments_Compounds_Plates_parsed_v4_all_new_measurements.csv"
clones_metadata_path = "/research/lab_kubicek/jreinis/Opera_images/2023-06-15_HAP1_cancer_41_clones_large_pooled_screen_repeat_suspicousplate/pool_of_41_cleaned_metadata_newOpera.csv"

## Pipeline config

In [ ]:
processed_files_folder_root = "/nobackup/lab_kubicek/jreinis/Opera_images_processed"
tmp_folder_root = "/nobackup/lab_kubicek/jreinis/batch_processing_temp_files"

In [ ]:
# channel name, desired number, number in dataset
channels_order = [
    ("GFP", 1, 3),
    ("mScarlet", 2, 5),
    ("BFP", 3, 1),
    ("mAmetrine", 4, 2),
    ("miRFP", 5, 4),
]

# suffixes of 1:1 mapping output masks
mapped_1_1_suffixes = {
    "filtered_cells": "_filtered_cells.png",
    "filtered_cytoplasm": "_filtered_cytoplasm.png",
    "filtered_nuclei": "_filtered_nuclei.png",
}

# default cluster queue
cluster_queue = "shortq"

# FFC-correction and separation settings
zip_preFFC = True    # create a backup of the files before FFC correction
zip_postFFC = True   # create a backup of the FFC-corrected files
delete_flex = True   # delete the original folder with FFC_corrected images after copying them
rename_utility_path = "/cm/shared/specific/apps/util-linux/2.36-GCCcore-10.2.0/bin/rename" # path to rename script from linux-utils: `rename old_pattern new_pattern files`
separate_FFC_script_body = "/home/jreinis/pipeline_scripts/separate_FFC_zip_partial.sh"

# slurm-specific settings
maximum_memory = "35000"
max_arr_jobs = 1000

# parameters for JPEG conversion
cluster_queue_JPEG = "shortq"
run_JPEG_conversion_script = "/home/jreinis/pipeline_scripts/run_JPEG_conversion.py"
extract_separate_chans = False
JPEG_quality="92" 
JPEG_resize_res="1080"
background_intensities = [0, 0, 0, 0, 0] # background intensity value for each channel - is subtracted for the visualization purposes
minq = 0.05                              # minimum quantile parameter for quantile normalization of the images
maxq = 0.9975                            # maximum quantile parameter for quantile normalization of the images
gamma = 1.00                             # gamma parameter for the visualization

# segmentatation and 1:1 mapping parameters
segmentation_batch_size = 20
cluster_queue_segmentation = "tinyq"
cellpose_script = "/home/jreinis/pipeline_scripts/run_cellpose_batch.py"
nucleaizer_script = "/home/jreinis/pipeline_scripts/run_nucleAIzer_batch_bleedthrough_correction.py"
mapping_script = "/home/jreinis/pipeline_scripts/run_1_1_mapping.py"
nucleaizer_model_path = "/home/jreinis/saved_models/mask_rcnn_general.h5"
colormap_path = "/home/jreinis/cellprofiler_pipelines/2022-03-18-glasbey_light.npy"
generate_inspection_imgs = False
nuclei_min_size = 750             # discard nuclei below area in px
cells_min_size = 1500             # discard cells below area in px
nuclei_overlap_min = 0.66         # minimal proportion of the area of a nucleus to lie within a cell to be assigned to it
NC_threshold_lower = 0.2          # discard cells below nuclear:cytoplasmic area threshold
NC_threshold_upper = 0.65         # discard cells above nuclear:cytoplasmic area threshold
FOV_border_px = 2                 # width of band around the FOV border, nuclei/cells within this area will be discarded
expand_px = 5                     # how many pixels to expand the mask of a cell when looking for neighboring ones
noborder_strict_mode = "chilled"  # strict mode ("strict") = discard CELLS touching border, non-strict mode = discard cells with NUCLEI touching border
apopt_thres_nuclei = 2000         # apply stricter filtering criteria to cells with nuclei below this size
apopt_thres_cells = 5000          # apply stricter filtering criteria to cells below this size
solidity_threshold = 0.95         # discard (apoptotic) cells with solidity above threshold
eccentricity_threshold = 1.4      # discard (apoptotic) cells with sum of cell+nucleus eccentricity above threshold

# segmented cell counting
cluster_queue_counting = "tinyq"
cell_counting_script = "/home/jreinis/pipeline_scripts/count_segmented_cells.py"
    
# cellprofiler parameters
cluster_queue_cellprofiler = "tinyq"
cellprofiler_batch_size = 30
cellprofiler_pipeline = "/home/jreinis/cellprofiler_pipelines/2022-02-18_CLUSTER_5channel_v7_no_1_1_mapping.cppipe"
cellprofiler_batch_merging_script = "/home/jreinis/pipeline_scripts/merge_cellprofiler_batches.py"
index_xml_filename = "index.xml"  # contains positions of FOVs, different Harmony software versions have different names
opera_harmony_version = "V6"      # different Harmony software versions have a slightly different structure of the index XML file

# saving cellprofiler/random forest predictions
cluster_queue_get_predictions = "tinyq"
get_rf_predictions_script = "/home/jreinis/pipeline_scripts/get_predictions_cellprofiler_rf.py"
rf_model_path = "/home/jreinis/saved_models/2023-03-28_HAP1_cancer_41_clones_newOpera_nomix_additional_E07all.joblib"

# visualizing cellprofiler/random forest predictions
cluster_queue_visualize_predictions = "shortq"
visualize_predictions_script = "/home/jreinis/pipeline_scripts/get_visualizations_predictions.py"
color_code_probabilities = 1

# post-perturbation detection parameters
cluster_queue_post_perturb_detection = "shortq"
extract_spatial_info_script = "/home/jreinis/pipeline_scripts/get_spatial_info.py"
combine_predictions_script = "/home/jreinis/pipeline_scripts/combine_predictions_spatial.py"
radius = 350      # radius in pixels to use to when fetching the neighboring cells
rf_models_channels_subsets = {
    "all_but_GFP": "/home/jreinis/saved_models/2023-03-28_HAP1_cancer_41_clones_newOpera_nomix_additional_E07all_but_GFP.joblib",
    "all_but_mScarlet": "/home/jreinis/saved_models/2023-03-28_HAP1_cancer_41_clones_newOpera_nomix_additional_E07all_but_mScarlet.joblib",
    "BFP_only": "/home/jreinis/saved_models/2023-03-28_HAP1_cancer_41_clones_newOpera_nomix_additional_E07BFP_only.joblib",
    "BFP_structural": "/home/jreinis/saved_models/2023-03-28_HAP1_cancer_41_clones_newOpera_nomix_additional_E07BFP_structural.joblib",
}

# hit calling parameters
cluster_queue_hit_calling = "tinyq"
features_to_test_path = "/home/jreinis/cellprofiler_pipelines/2023-04-13_hit_calling_features_use_test.csv"
test_statistic_cellprofiler_script = "/home/jreinis/pipeline_scripts/tests_statistics_cellprofiler_features.py"

# visualizing timecourses
visualize_timecourses_script_path = "/home/jreinis/pipeline_scripts/visualize_predictions_timecourse.py"
size_tile_overlay = 10
show_labels = 1
color_code_probas = 1
generate_separate_channels = 1

In [ ]:
#### prepare renumbering of different channels
channels_order = pd.DataFrame(channels_order, columns = ["name", "analysis_index", "dataset_index"])
chan_suffixes = {name:f"-ch{ch}sk1fk1fl1.tiff" for name,ch in zip(channels_order["name"],channels_order["analysis_index"])}
channels_order

In [ ]:
#### Save processed run folders in a variable
processed_runs = [f"{processed_files_folder_root}/{analysis_name}/{get_run_name(run)}_FFC" for run in Opera_runs]

## Prepare folders and variables
1. create folder named `analysis_name` in `processed_files_folder`
2. for each processed measurement create a folder named `[measurement name from Opera] + "_FFC"`
3. store all processed files + scripts + logs there

In [ ]:
processed_files_folder = f"{processed_files_folder_root}/{analysis_name}"
tmp_files_folder = f"{tmp_folder_root}/{analysis_name}"
scripts_folder = f"{processed_files_folder}/scripts"
logs_folder = f"{processed_files_folder}/logs"
hit_calling_folder = f"{processed_files_folder}/hit_calling"
analysis_metadata_folder = f"{processed_files_folder}/metadata"

os.system(f"mkdir -p {processed_files_folder} {tmp_files_folder} {scripts_folder} {logs_folder}")
print(f"creating root folders for analysis: {analysis_name}\n {processed_files_folder}\n {scripts_folder}\n {tmp_files_folder}\n")
print(f"creating subfolders for measurements") 

for run_folder_FFC in processed_runs:
    # extract the name of the Opera run from the filepath
    run_name = get_run_name(run_folder_FFC)

    # output files go here
    metadata_folder = f"{run_folder_FFC}/Images_metadata/"
    output_folder_tagged_JPEG = f"{run_folder_FFC}/Images_tagged_chans_JPEG/"
    output_folder_structural_JPEG = f"{run_folder_FFC}/Images_structural_chans_JPEG/"
    output_folder_channels_JPEG = f"{run_folder_FFC}/Images_separate_chans_JPEG/"
    output_folder_segmented = f"{run_folder_FFC}/Images_segmented/"
    output_folder_segmented_inspect = f"{run_folder_FFC}/Images_segmented_inspect/"
    output_folder_cellprofiler = f"{run_folder_FFC}/CellProfiler_measurements/"
    output_folder_extracted_cells = f"{run_folder_FFC}/Images_extracted_cells/"
    output_folder_cytoself = f"{run_folder_FFC}/Cytoself_measurements/"
    output_folder_cellprofiler_predictions = f"{run_folder_FFC}/Predictions_CellProfiler_RF/"
    
    # generated bash scripts go here   
    scripts_folder_separateFFC = f"{scripts_folder}/{run_name}/00_FFC_separate/"
    scripts_folder_JPEG = f"{scripts_folder}/{run_name}/01_JPEG_convert/"
    scripts_folder_segmentation = f"{scripts_folder}/{run_name}/02_segmentation/"
    scripts_folder_counting = f"{scripts_folder}/{run_name}/03_count_segmented_cells/"
    scripts_folder_cellprofiler = f"{scripts_folder}/{run_name}/04_cellprofiler/"
    scripts_folder_get_cp_predictions = f"{scripts_folder}/{run_name}/05_cellprofiler_predictions/"
    scripts_folder_visualize_cp_predictions = f"{scripts_folder}/{run_name}/06_visualize_cellprofiler_predictions/"
    scripts_folder_extraction = f"{scripts_folder}/{run_name}/06_cell_extraction/"
    scripts_folder_cytoself = f"{scripts_folder}/{run_name}/07_cytoself/"

    tmp_folder_run = f"{tmp_files_folder}/{run_name}"
    # temp folder for intermediary files, e.g. cellprofiler measurements batches that need to be merged
    tmp_folder_cellprofiler = f"{tmp_folder_run}/cellprofiler_temp/"
    
    # run the folder creation commands
    os.system(f"mkdir -p {metadata_folder} {output_folder_tagged_JPEG} {output_folder_structural_JPEG} {output_folder_channels_JPEG}")
    os.system(f"mkdir -p {output_folder_segmented} {output_folder_segmented_inspect} {output_folder_cellprofiler} {output_folder_cellprofiler_predictions}")
#    os.system(f"mkdir -p {output_folder_extracted_cells} {output_folder_cytoself}")
   
    os.system(f"mkdir -p {scripts_folder_separateFFC} {scripts_folder_JPEG} {scripts_folder_segmentation} {scripts_folder_counting}")
    os.system(f"mkdir -p {scripts_folder_cellprofiler}")
    os.system(f"mkdir -p {scripts_folder_get_cp_predictions} {scripts_folder_visualize_cp_predictions}")
#    os.system(f"mkdir -p {scripts_folder_extraction} {scripts_folder_cytoself}")
    
    os.system(f"mkdir -p {tmp_folder_cellprofiler}")

    print(f" {run_name}")
print()
    
#### if hit calling: copy platemap and clones metadata to the analysis folder
# folder to save statistical tests results
if "platemap_path" in locals() and "clones_metadata_path" in locals():
    os.system(f"mkdir -p {hit_calling_folder}")
    os.system(f"mkdir -p {analysis_metadata_folder}")
    print("creating folders for hit calling", " "+analysis_metadata_folder, " "+hit_calling_folder, sep="\n", end="\n")
    
# copy platemap and clones_metadata to the analysis folder
if "platemap_path" in locals():
    platemap_path_cp = f'{analysis_metadata_folder}/{platemap_path.split("/")[-1]}'
    os.system(f'cp {platemap_path} {platemap_path_cp}')
    print(f"copying platemap to {processed_files_folder_root}/{analysis_name}")
if "clones_metadata_path" in locals():
    clones_metadata_path_cp = f'{analysis_metadata_folder}/{clones_metadata_path.split("/")[-1]}'
    os.system(f'cp {clones_metadata_path} {clones_metadata_path_cp}')
    print(f"copying clones metadata to {processed_files_folder_root}/{analysis_name}\n")

## 0. Separate FFC-corrected images, zip original for backup

In [ ]:
master_script_path = f'{scripts_folder}/{analysis_name}_00_separate_FFC_zip.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
            f.write("#!/bin/bash\n\n")

for run_folder, run_folder_FFC in zip(Opera_runs, processed_runs):
    run_name = get_run_name(run_folder_FFC)
    scripts_folder_separateFFC = f"{scripts_folder}/{run_name}/00_FFC_separate"
    script_path = f"{scripts_folder_separateFFC}/{run_name}_00_separate_FFC_zip.sh"
    
    zip_preFFC_param = f'{run_folder}_pre_FFC.zip' if zip_preFFC else "False"
    zip_postFFC_param = f'{run_folder}_post_FFC.zip' if zip_postFFC else "False"
    
    script_head = "\n".join([f"#!/bin/bash",
                             f"#SBATCH --output {logs_folder}/job_%j_{run_name}_separate_FFC_zip.log",
                             f"#SBATCH --error {logs_folder}/job_%j_{run_name}_separate_FFC_zip.log",
                             f"#SBATCH --job-name=FFC_sep",
                             f"#SBATCH --partition={cluster_queue}",
                             f"#SBATCH --qos={cluster_queue}\n",
                             f"source /home/${{USER}}/.bashrc\n",
                             "date; echo\n\n"])

    script_config = "\n".join([
        f'measurement="{run_folder}"',
        f'measurement_FFC="{run_folder_FFC}"',
        f'zip="{zip_preFFC_param}"',
        f'zip_FFC="{zip_postFFC_param}"',
        f'delete_flex="{delete_flex}"',
        f'rename_script="{rename_utility_path}"',
        f'channels_order=({" ".join(channels_order["dataset_index"].astype(str))})\n\n',
    ])
    
    # load the part of the script that is always the same
    with open(separate_FFC_script_body, mode="r", encoding="utf-8") as f:
        script_body = f.read()
    
    with open(script_path, mode='w', encoding='utf-8') as f:
        f.write(script_head)
        f.write(script_config)
        f.write(script_body)
    
    with open(master_script_path, mode='a', encoding='utf-8') as f:
        f.write(f"chmod ug+rx {script_path}\n")
        f.write(f"sbatch {script_path}\n\n")

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## Prepare imagelist metadata file
- metadata file with paths and filenames to raw and processed files for each FOV
- required for CellProfiler - dictates also the structure
- but used also by Cellpose scripts and cell extraction

In [ ]:
for run_folder_FFC in tqdm(processed_runs):
    run_name = get_run_name(run_folder_FFC)
    
    # csv path
    metadata_folder = f"{run_folder_FFC}/Images_metadata/"
    imagelist_csv_path = f"{metadata_folder}/{run_name}_images_list.csv"
    
    # folders
    raw_tif_folder = f"{run_folder_FFC}/Images_FFC/"
    output_folder_tagged_JPEG = f"{run_folder_FFC}/Images_tagged_chans_JPEG/"
    output_folder_structural_JPEG = f"{run_folder_FFC}/Images_structural_chans_JPEG/"
    output_folder_segmented = f"{run_folder_FFC}/Images_segmented/"
    output_folder_extracted_cells = f"{run_folder_FFC}/Images_extracted_cells/"

    # load list of images (FOVs)
    FOVs = list(sorted(set([x.split("-")[0] for x in os.listdir(raw_tif_folder) if x.endswith(".tiff")])))

    # variable to store all the metadata in a dictionary
    imagelist = {}
      
    # prepare the fields in the csv file
    # FOV IDs (e.g. r01c01f04p01)
    imagelist["FOV"] = FOVs
    
    # required for CellProfiler?
    imagelist["Group_Number"] = 1
    
    # raw image files - 5 files per FOV, for each path + filename + URL (="file:/"+path+filename)
    for chan, suffix in chan_suffixes.items():
        imagelist["FileName_"+chan] = [img+suffix for img in FOVs]
        imagelist["PathName_"+chan] = raw_tif_folder
        imagelist["URL_"+chan] = ["file:"+raw_tif_folder+filename for filename in imagelist["FileName_"+chan]]

    # tagged/structural channels
    imagelist["FileName_tagged_chans"] = [img + "_tagged_channels.jpg" for img in FOVs]
    imagelist["PathName_tagged_chans"] = output_folder_tagged_JPEG
    imagelist["URL_tagged_chans"] = ["file:"+output_folder_tagged_JPEG+filename for filename in imagelist["FileName_tagged_chans"]]
    imagelist["FileName_structural_chans"] = [img + "_structural_channels.jpg" for img in FOVs]
    imagelist["PathName_structural_chans"] = output_folder_structural_JPEG
    imagelist["URL_structural_chans"] = ["file:"+output_folder_structural_JPEG+filename for filename in imagelist["FileName_structural_chans"]]
    
    # segmented output names
    imagelist["FileName_segmented_cells_mask"] = [img + chan_suffixes["mAmetrine"][:-5] + "_segmented_mask.png" for img in FOVs]
    imagelist["PathName_segmented_cells_mask"] = output_folder_segmented
    imagelist["URL_segmented_cells_mask"] = ["file:"+output_folder_segmented+filename for filename in imagelist["FileName_segmented_cells_mask"]]
    imagelist["FileName_segmented_nuclei_mask"] = [img + chan_suffixes["miRFP"][:-5] + "_segmented_mask.png" for img in FOVs]
    imagelist["PathName_segmented_nuclei_mask"] = output_folder_segmented
    imagelist["URL_segmented_nuclei_mask"] = ["file:"+output_folder_segmented+filename for filename in imagelist["FileName_segmented_nuclei_mask"]]

    # 1:1 mapping output: cells, nuclei, cytoplasm
    for component, suffix in mapped_1_1_suffixes.items():
        imagelist["FileName_"+component] = [img+suffix for img in FOVs]
        imagelist["PathName_"+component] = output_folder_segmented
        imagelist["URL_"+component] = ["file:"+output_folder_segmented+filename for filename in imagelist["FileName_"+component]]
        
    # extracted masks
    imagelist["PathName_extracted_cells"] = output_folder_extracted_cells
    imagelist["FileName_extracted_cells"] = [img + "_extracted_cells.h5" for img in FOVs]
    imagelist["URL_extracted_cells"] = ["file:"+output_folder_extracted_cells+filename for filename in imagelist["FileName_extracted_cells"]]
          
    # cytoself
    imagelist["PathName_cytoself"] = output_folder_cytoself
    imagelist["FileName_cytoself"] = [img + "_cytoself_features.h5" for img in FOVs]
    imagelist["URL_cytoself"] = ["file:"+output_folder_cytoself+filename for filename in imagelist["FileName_cytoself"]]

    imagelist = pd.DataFrame(imagelist)
    # Number the FOVs (required for batch processing)
    imagelist = imagelist.reset_index()
    imagelist.rename(columns={"index": "Group_Index"}, inplace=True)
    # 1-based indexing for CellProfiler
    imagelist["Group_Index"] = imagelist["Group_Index"] + 1
    
    # columns need to be in specific order (?) for CellProfiler, otherwise it does not seem to work
    new_col_order = ["Group_Number", "Group_Index"]
    # add file/path names
    columns_URLs = [col for col in imagelist.columns if col.startswith("URL_")]
    columns_filenames = [col for col in imagelist.columns if col.startswith("FileName_")]
    columns_pathnames = [col for col in imagelist.columns if col.startswith("PathName_")]
    for cols in zip(columns_URLs, columns_filenames, columns_pathnames):
            new_col_order.extend([cols[0], cols[1], cols[2]])
    # FOV name
    new_col_order += ["FOV"]
    
    # save reordered
    imagelist[new_col_order].to_csv(imagelist_csv_path, index=None)

## 1. JPEG conversion scripts

In [ ]:
# JPEG conversion - add flag if 
sep_chans_par = {False:"", True:"--separate "}[extract_separate_chans]

master_script_path = f'{scripts_folder}/{analysis_name}_01_run_JPEG_conversion.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")

for run_folder_FFC in processed_runs:
    run_name = get_run_name(run_folder_FFC)
    scripts_folder_JPEG = f"{scripts_folder}/{run_name}/01_JPEG_convert"
    script_path = f"{scripts_folder_JPEG}/{run_name}_JPEG_convert.sh"

    script_head = "\n".join([f"#!/bin/bash",
                             f"#SBATCH --output {logs_folder}/job_%j_{run_name}_JPEG_convert.log",
                             f"#SBATCH --error {logs_folder}/job_%j_{run_name}_JPEG_convert.log",
                             "#SBATCH --job-name=convert",
                             f"#SBATCH --partition={cluster_queue_JPEG}",
                             f"#SBATCH --qos={cluster_queue_JPEG}\n",
                             f"source /home/${{USER}}/.bashrc\n",
                             "date; echo\n\n"])

    script_body = "\n".join([f"measurement=\"{run_folder_FFC}/\"",
                             f"quality={JPEG_quality}",
                             f"resolution={JPEG_resize_res}\n",
                             f"gamma={gamma}\n",
                             f'{run_JPEG_conversion_script} -m $measurement -r $resolution -q $quality {sep_chans_par} -g $gamma -b {" ".join([str(x) for x in background_intensities])}\n',
                             "echo; date",
                             "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])
    
    with open(script_path, mode='w', encoding='utf-8') as f:
        f.write(script_head)
        f.write(script_body)
    
    with open(master_script_path, mode='a', encoding='utf-8') as f:
        f.write(f"chmod ug+rx {script_path}\n")
        f.write(f"sbatch {script_path}\n\n")

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## 2. Segmentation scripts
1. **cellpose** (cells)
2. **nucleAIzer** (nuclei)
3. **custom scripts** (1:1 mapping + filtering)

In [ ]:
master_script_path = f'{scripts_folder}/{analysis_name}_02_run_segmentation.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")

for run_folder_FFC in processed_runs:
    run_name = get_run_name(run_folder_FFC)
    # paths
    scripts_folder_segmentation = f"{scripts_folder}/{run_name}/02_segmentation/"
    metadata_folder = f"{run_folder_FFC}/Images_metadata/"
    imagelist_csv_path = f"{metadata_folder}/{run_name}_images_list.csv"
    output_folder_segmented_inspect = f"{run_folder_FFC}/Images_segmented_inspect/"

    # load imagelist
    imagelist = pd.read_csv(imagelist_csv_path)

    n_batches = np.ceil(len(imagelist)/segmentation_batch_size).astype("int")
    
    # prepare a script to run the segmentation in batches (using arrayed slurm jobs)
    script_path = f"{scripts_folder_segmentation}/{run_name}_02_run_segmentation.sh"

    # create the script - the array line ensures one script is run for each batch
    script_head = "\n".join([f"#!/bin/bash",
                             f"#SBATCH --output {logs_folder}/job_%A_%3a_{run_name}_segmentation.log",
                             f"#SBATCH --error {logs_folder}/job_%A_%3a_{run_name}_segmentation.log",
                             f"#SBATCH --mem={maximum_memory}",
                             f"#SBATCH --job-name=segment",
                             f"#SBATCH --partition={cluster_queue_segmentation}",
                             f"#SBATCH --qos={cluster_queue_segmentation}",
                             f"#SBATCH --array=0-{max(range(n_batches))}:1%{max_arr_jobs}\n", 
                             "source /home/${USER}/.bashrc\n",
                             "date; echo\n",
                             f"echo {run_name}\n",
                             'batch_i=$SLURM_ARRAY_TASK_ID',
                             f'batch_size={segmentation_batch_size}\n',
                             'start=$(( batch_i*batch_size ))',
                             'stop=$(( (batch_i+1)*batch_size ))\n\n'])#echo $batch_i $start $stop\n'])
    
    # config
    script_body = "\n".join([f'imagelist={imagelist_csv_path}\n\n'])
    
    # cellpose part (cells)
    script_body += "\n".join([f"conda activate cellpose",
                             f"{cellpose_script} -i $imagelist -f $start -l $stop\n\n"])
    
    # nucleaizer part (nuclei)
    script_body += "\n".join([f"conda activate nucleaizer",
                             f'{nucleaizer_script} -i $imagelist -m {nucleaizer_model_path} -f $start -l $stop -n {minq} -x {maxq} -b {" ".join([str(x) for x in background_intensities])}\n',
                             "echo; date\n"]) 
    
    # 1:1 mapping part
    mapping_script_command = f'{mapping_script} -i $imagelist -f $start -l $stop -Nm {nuclei_min_size} -Cm {cells_min_size} -o {nuclei_overlap_min} -L {NC_threshold_lower} -U {NC_threshold_upper} -b {FOV_border_px} '
    if generate_inspection_imgs:
            mapping_script_command += f'-O {output_folder_segmented_inspect} '
    mapping_script_command += f'-s {noborder_strict_mode} -e {expand_px} -c {colormap_path} -Na {apopt_thres_nuclei} -Ca {apopt_thres_cells} -S {solidity_threshold} -E {eccentricity_threshold}'
    script_body += "\n".join([f"conda activate base",
                             mapping_script_command,
                             "echo; date",
                             "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])
    #print(script_path)
    # save
    with open(script_path, mode='w', encoding='utf-8') as f:
        f.write(script_head)
        f.write(script_body)
            
    with open(master_script_path, mode='a', encoding='utf-8') as f:
        f.write(f"chmod ug+rx {script_path}\n")
        f.write(f"sbatch {script_path}\n\n")

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## 3. Count segmented cells

In [ ]:
master_script_path = f'{scripts_folder}/{analysis_name}_03_count_segmented_cells.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")

for run_folder_FFC in processed_runs:
    run_name = get_run_name(run_folder_FFC)
    scripts_folder_counting = f"{scripts_folder}/{run_name}/03_count_segmented_cells/"
    script_path = f"{scripts_folder_counting}{run_name}_03_count_segmented_cells.sh"
    
    # define folders/parameters to pass on to script
    metadata_folder = f"{run_folder_FFC}/Images_metadata/"
    imagelist_csv_path = f"{metadata_folder}/{run_name}_images_list.csv"
    cell_counts_csv_path = f"{metadata_folder}/{run_name}_cell_counts.csv"
       
    script_head = "\n".join([f"#!/bin/bash",
                             f"#SBATCH --output {logs_folder}/job_%j_{run_name}_count_segmented_cells.log",
                             f"#SBATCH --error {logs_folder}/job_%j_{run_name}_count_segmented_cells.log",
                             "#SBATCH --job-name=count_cells",
                             f"#SBATCH --partition={cluster_queue_counting}",
                             f"#SBATCH --qos={cluster_queue_counting}\n",
                             f"source /home/${{USER}}/.bashrc\n",
                             "date; echo\n\n"])

    script_body = "\n".join([f'{cell_counting_script} -i {imagelist_csv_path} -o {cell_counts_csv_path} -r {run_name} -Nm {nuclei_min_size} -Cm {cells_min_size}\n',
                             "echo; date",
                             "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])

    #print(script_path)
    with open(script_path, mode='w', encoding='utf-8') as f:
        f.write(script_head)
        f.write(script_body)
    
    with open(master_script_path, mode='a', encoding='utf-8') as f:
        f.write(f"chmod ug+rx {script_path}\n")
        f.write(f"sbatch {script_path}\n\n")

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## 4. CellProfiler scripts
1. **extract CellProfiler measurements for filtered cells, nuclei, and cytoplasm objects**
    - perform in batches
    - needs a (column) subset of the Images_list.csv file, without extracted cell paths (and unprocessed cellpose output):
        - named `Images_list_CellProfiler.csv`, saved in the main measurement folder
2. **merge batches**
- note: for some reason calling the scripts from this notebook fails to load the CellProfiler module

In [ ]:
# helper function, used in excluding columns in imagelist file not to pass to cellprofiler
def contains_pattern(string, patterns):
    for pattern in patterns:
        if pattern in string:
            return True
    return False

master_script_path = f'{scripts_folder}/{analysis_name}_04_cellprofiler.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")

for run_folder_FFC in processed_runs:
    run_name = get_run_name(run_folder_FFC)
    # paths
    scripts_folder_cellprofiler = f"{scripts_folder}/{run_name}/04_cellprofiler/"
   
    # temp folder for intermediary files - batches that need to be merged
    tmp_folder_cellprofiler = f"{tmp_files_folder}/{run_name}/cellprofiler_temp/"
    output_folder_cellprofiler = f"{run_folder_FFC}/CellProfiler_measurements/"

    metadata_folder = f"{run_folder_FFC}/Images_metadata/"
    imagelist_csv_path = f"{metadata_folder}/{run_name}_images_list.csv"
    imagelist_csv_cellprofiler_path = f"{metadata_folder}/{run_name}_images_list_CellProfiler.csv"
    index_xml_path = f"{run_folder_FFC}/Images_FFC/{index_xml_filename}"
       
    # generate imagelist subset
    imagelist = pd.read_csv(imagelist_csv_path)
    # define columns not to pass on to CellProfiler so that it does not load them
    cols_keep = [col for col in imagelist.columns if not contains_pattern(col, ["extracted_cells", "cellpose", "cytoself", "structural", "tagged"])]
    # save selected columns in a cellprofiler-specific imagelist
    imagelist[cols_keep].to_csv(imagelist_csv_cellprofiler_path, index=None)
    
    n_batches = np.ceil(len(imagelist)/cellprofiler_batch_size).astype("int")

    script_path = f"{scripts_folder_cellprofiler}/{run_name}_04_a_run_cellprofiler.sh"
    # create the script - the array line ensures one script is run for each batch
    script_head = "\n".join([f"#!/bin/bash",
                             f"#SBATCH --output {logs_folder}/job_%A_%3a_{run_name}_cellprofiler.log",
                             f"#SBATCH --error {logs_folder}/job_%A_%3a_{run_name}_cellprofiler.log",
                             f"#SBATCH --mem={maximum_memory}",
                             f"#SBATCH --job-name=cellprofiler",
                             f"#SBATCH --partition={cluster_queue_cellprofiler}",
                             f"#SBATCH --qos={cluster_queue_cellprofiler}",
                             f"#SBATCH --array=0-{max(range(n_batches))}:1%{max_arr_jobs}\n", 
                             "source /home/${USER}/.bashrc\n",
                             "module load CellProfiler/4.2.1-foss-2020b\n\n",                            
                             "date; echo\n",
                             f"echo {run_name}\n",
                             'batch_i=$SLURM_ARRAY_TASK_ID',
                             f'batch_size={cellprofiler_batch_size}\n',
                             f'maxa={len(imagelist)}',
                             'start=$(( batch_i*batch_size +1 ))',
                             'stop=$(( (batch_i+1)*batch_size ))',
                             'if [ $stop -ge $maxa ]; then stop=$maxa; fi',
                             f'printf -v tmp_folder_batch "{tmp_folder_cellprofiler}/batch_%04d" $batch_i',
                             'echo $batch_i $start $stop',
                             'echo $tmp_folder_batch\n',
                             ])

    script_body = "\n".join([f"cellprofiler -c -r -p {cellprofiler_pipeline} --data-file {imagelist_csv_cellprofiler_path} -f $start -l $stop -o $tmp_folder_batch\n",
                  "echo; date",
                  "sstat  -j   $SLURM_JOB_ID.batch   --format=JobID,Node,MaxVMSize\n"])
    
    with open(script_path, mode='w', encoding='utf-8') as f:
        f.write(script_head)
        f.write(script_body)
        
        
    # merging of cellprofiler batches script
    merging_script_path = f"{scripts_folder_cellprofiler}/{run_name}_04_b_merge_cellprofiler_batches.sh"

    merging_script_head = "\n".join([f"#!/bin/bash",
                                     f"#SBATCH --output {logs_folder}/job_%j_{run_name}_merge_CP_batches.log",
                                     f"#SBATCH --error {logs_folder}/job_%j_{run_name}_merge_CP_batches.log",
                                     f"#SBATCH --job-name=merge_CP",
                                     f"#SBATCH --partition={cluster_queue}",
                                     f"#SBATCH --qos={cluster_queue}\n",
                                     f"source /home/${{USER}}/.bashrc\n",
                                     "date; echo\n\n"])

    merging_script_body = "\n".join([f'imagelist="{imagelist_csv_path}"',
                                     f'index_xml_path="{index_xml_path}"',
                                     f'batches_folder_root="{tmp_folder_cellprofiler}"',
                                     f'output_folder="{output_folder_cellprofiler}"\n',
                                     f'{cellprofiler_batch_merging_script} -i $imagelist -x $index_xml_path -b $batches_folder_root -o $output_folder -v {opera_harmony_version}\n',
                                     "echo; date",
                                     "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])

    with open(merging_script_path, mode='w', encoding='utf-8') as f:
        f.write(merging_script_head)
        f.write(merging_script_body)
    
    # master script - include dependency of merging batches on running cellprofiler first
    with open(master_script_path, mode='a', encoding='utf-8') as f:
        f.write(f"jid=$(sbatch --parsable {script_path})\n")
        f.write(f"chmod ug+rx {merging_script_path}\n")          
        f.write(f"sbatch --dependency=afterok:${{jid}} {merging_script_path}\n\n")

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## 5. Get random forest predictions

In [ ]:
master_script_path = f'{scripts_folder}/{analysis_name}_05_get_CellProfiler_predictions.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")
            
for run_folder_FFC in processed_runs:
    run_name = get_run_name(run_folder_FFC)
    # paths
    scripts_folder_get_cp_predictions = f"{scripts_folder}/{run_name}/05_cellprofiler_predictions/"

    cellprofiler_measurements_path = f"{run_folder_FFC}/CellProfiler_measurements/{run_name}_merged_cell_measurements.csv"
    output_path_cellprofiler_predictions = f"{run_folder_FFC}/Predictions_CellProfiler_RF/{run_name}_predictions_cellprofiler_rf.csv"
    
    script_path = f"{scripts_folder_get_cp_predictions}/{run_name}_05_get_CellProfiler_predictions.sh"

    script_head = "\n".join([f"#!/bin/bash",
                             f"#SBATCH --output {logs_folder}/job_%j_{run_name}_get_predictions.log",
                             f"#SBATCH --error {logs_folder}/job_%j_{run_name}_get_predictions.log",
                             "#SBATCH --job-name=get_predictions",
                             f"#SBATCH --partition={cluster_queue_get_predictions}",
                             f"#SBATCH --qos={cluster_queue_get_predictions}\n",
                             f"source /home/${{USER}}/.bashrc\n",
                             "date; echo\n\n"])

    script_body = "\n".join([f"measurement=\"{cellprofiler_measurements_path}\"",
                             f"rf_model_path=\"{rf_model_path}\"",
                             f"predictions_save_path={output_path_cellprofiler_predictions}\n",
                             f'{get_rf_predictions_script} -i $measurement -m $rf_model_path -o $predictions_save_path\n',
                             "echo; date",
                             "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])
    
    with open(script_path, mode='w', encoding='utf-8') as f:
        f.write(script_head)
        f.write(script_body)
    
    with open(master_script_path, mode='a', encoding='utf-8') as f:
        f.write(f"chmod ug+rx {script_path}\n")
        f.write(f"sbatch {script_path}\n\n")

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## 6. Visualize random forest predictions

In [ ]:
# JPEG conversion - add flag if 
color_code_par = {False:"", True:"--color_code_probas "}[bool(color_code_probabilities)]

master_script_path = f'{scripts_folder}/{analysis_name}_06_visualize_cellprofiler_predictions.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")

for run_folder_FFC in processed_runs:
    run_name = get_run_name(run_folder_FFC)
    
    scripts_folder_visualize_cp_predictions = f"{scripts_folder}/{run_name}/06_visualize_cellprofiler_predictions/"

    # define folders/parameters to pass on to script
    metadata_folder = f"{run_folder_FFC}/Images_metadata/"
    imagelist_csv_path = f"{metadata_folder}/{run_name}_images_list.csv"
    cellprofiler_predictions_path = f"{run_folder_FFC}/Predictions_CellProfiler_RF/{run_name}_predictions_cellprofiler_rf.csv"
    
    output_folder_visualizations = f"{run_folder_FFC}/Predictions_CellProfiler_RF/"
    output_folder_comparison = f"{output_folder_visualizations}/comparisons"
    output_folder_tagged_labeled = f"{output_folder_visualizations}/labeled_tagged"
    
    background_intensities_param = " ".join([str(x) for x in background_intensities])
    
    script_path = f"{scripts_folder_visualize_cp_predictions}/{run_name}_06_visualize_cellprofiler_predictions.sh"

    script_head = "\n".join([f"#!/bin/bash",
                             f"#SBATCH --output {logs_folder}/job_%j_{run_name}_visualize_predictions.log",
                             f"#SBATCH --error {logs_folder}/job_%j_{run_name}_visualize_predictions.log",
                             f"#SBATCH --mem={maximum_memory}",                            
                             f"#SBATCH --job-name=visualize_pred",
                             f"#SBATCH --partition={cluster_queue_visualize_predictions}",
                             f"#SBATCH --qos={cluster_queue_visualize_predictions}\n",
                             f"source /home/${{USER}}/.bashrc\n",
                             "date; echo\n\n"])

    script_body = "\n".join([f'imagelist="{imagelist_csv_path}"',
                             f'predictions="{cellprofiler_predictions_path}"',
                             f'rf_model_path="{rf_model_path}"\n',
                             f'output_comparison="{output_folder_comparison}"',
                             f'output_tagged="{output_folder_tagged_labeled}"\n',
                             f'mkdir -p $output_comparison $output_tagged\n',
                             f'{visualize_predictions_script} -i $imagelist -p $predictions -r $rf_model_path -C $output_comparison -t $output_tagged -b {background_intensities_param} -n {minq} -x {maxq} -g {gamma} -Nm {nuclei_min_size} -Cm {cells_min_size} -c {colormap_path} {color_code_par} \n',
                             "echo; date",
                             "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])
    
    with open(script_path, mode='w', encoding='utf-8') as f:
        f.write(script_head)
        f.write(script_body)
    
    with open(master_script_path, mode='a', encoding='utf-8') as f:
        f.write(f"chmod ug+rx {script_path}\n")
        f.write(f"sbatch {script_path}\n\n")

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## 90. Get post-perturbation predictions (random forest chans subset + spatial info)

In [ ]:
# group the measurements by plate name; convention: pretreatment is always measurement 1
runs = pd.DataFrame(processed_runs, columns=["run_folder"])
runs["run_name"] = [get_run_name(path) for path in processed_runs]
runs["plate"] = [x.split("__")[0] for x in runs["run_name"]]
runs["measurement"] = [int(x.split("-")[-1].strip("_FFC").split("_")[1]) for x in runs["run_name"]]
runs["cellprofiler_path"] = [f'{runs.loc[i, "run_folder"]}/CellProfiler_measurements/{runs.loc[i, "run_name"]}_merged_cell_measurements.csv' for i in range(len(runs))]
runs["predictions_path"] = [f'{runs.loc[i, "run_folder"]}/Predictions_CellProfiler_RF/{runs.loc[i, "run_name"]}_predictions_cellprofiler_rf.csv' for i in range(len(runs))]

plates = sorted(runs.plate.unique())

In [ ]:
master_script_path = f'{scripts_folder}/{analysis_name}_90_get_post_perturb_predictions.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")

for plate in tqdm(plates):
    runs_plate = runs[runs["plate"] == plate]
    
    # first measurement
    pretreatment_idx = runs_plate[runs_plate.measurement == 1].index[0]
    pretreatment_preds_path = runs_plate.loc[pretreatment_idx, "predictions_path"]
    # all other treatments
    treat_idxs = runs_plate[runs_plate.index != pretreatment_idx].index
    
    # generate the necessary scripts for all post-treatment measurements
    for idx in treat_idxs:
        run_folder_FFC = runs.loc[idx, "run_folder"]
        run_name = get_run_name(run_folder_FFC)
        
        scripts_folder_get_predictions_post_perturb = f"{scripts_folder}/{run_name}/90_get_predictions_post_perturb"
        os.system(f"mkdir -p {scripts_folder_get_predictions_post_perturb}")
             
        ## get spatial info predictions
        cellprofiler_treated_path = runs.loc[idx, "cellprofiler_path"]
        spatial_preds_output_path = f'{run_folder_FFC}/Predictions_CellProfiler_RF/{run_name}_spatial_info_scores.csv'
        
        script_path = f"{scripts_folder_get_predictions_post_perturb}/{run_name}_90a_get_spatial_info.sh"

        script_head = "\n".join([f"#!/bin/bash",
                                 f"#SBATCH --output {logs_folder}/job_%j_{run_name}_get_spatial_info.log",
                                 f"#SBATCH --error {logs_folder}/job_%j_{run_name}_get_spatial_info.log",
                                 f"#SBATCH --mem={maximum_memory}",                                 
                                 f"#SBATCH --job-name=spatial_preds",
                                 f"#SBATCH --partition={cluster_queue_post_perturb_detection}",
                                 f"#SBATCH --qos={cluster_queue_post_perturb_detection}\n",
                                 f"source /home/${{USER}}/.bashrc\n",
                                 "date; echo\n\n"])
        
        script_body = "\n".join([f'cells_T1_path="{cellprofiler_treated_path}"',
                                 f'predictions_T0_path="{pretreatment_preds_path}"',
                                 f'output_path="{spatial_preds_output_path}"',
                                 f'radius={radius}\n',
                                 f'{extract_spatial_info_script} -c $cells_T1_path -p $predictions_T0_path -o $output_path -r $radius\n',
                                 "echo; date",
                                 "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])
        
        with open(script_path, mode='w', encoding='utf-8') as f:
            f.write(script_head)
            f.write(script_body)
        # master script - prepare dependencies
        with open(master_script_path, mode='a', encoding='utf-8') as f:
            f.write(f"# {run_name}\n")
            f.write(f"jids=$(sbatch --parsable {script_path})\n")
            
        ## get channel subsets model predictions
        channel_subsets_preds_output_path_basename = f'{run_folder_FFC}/Predictions_CellProfiler_RF/{run_name}_predictions_cellprofiler_rf_channelsubset'

        for i, (rf_name, rf_path) in enumerate(rf_models_channels_subsets.items()):
            script_path = f"{scripts_folder_get_predictions_post_perturb}/{run_name}_90b_get_CellProfiler_predictions_channel_subsets_{i:02d}_{rf_name}.sh"
            predictions_output_path = f"{channel_subsets_preds_output_path_basename}_{i:02d}_{rf_name}.csv"

            script_head = "\n".join([f"#!/bin/bash",
                                     f"#SBATCH --output {logs_folder}/job_%j_{run_name}_get_predictions_channelsubsets_{i:02d}_{rf_name}.log",
                                     f"#SBATCH --error {logs_folder}/job_%j_{run_name}_get_predictions_channelsubsets_{i:02d}_{rf_name}.log",
                                     f"#SBATCH --mem={maximum_memory}",                                 
                                     f"#SBATCH --job-name=get_predictions",
                                     f"#SBATCH --partition={cluster_queue_get_predictions}",
                                     f"#SBATCH --qos={cluster_queue_get_predictions}\n",
                                     f"source /home/${{USER}}/.bashrc\n",
                                     "date; echo\n\n"])

            script_body = "\n".join([f"measurement=\"{cellprofiler_treated_path}\"",
                                     f"rf_model_path=\"{rf_path}\"",
                                     f"predictions_save_path={predictions_output_path}\n",
                                     f'{get_rf_predictions_script} -i $measurement -m $rf_model_path -o $predictions_save_path\n',
                                     "echo; date",
                                     "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])

            with open(script_path, mode='w', encoding='utf-8') as f:
                f.write(script_head)
                f.write(script_body)
                
            # master script - prepare dependencies
            with open(master_script_path, mode='a', encoding='utf-8') as f:
                f.write(f"jids=$jids,$(sbatch --parsable {script_path})\n")
                
        
        ## final script combining the spatial info with channel subsets
        script_path = f"{scripts_folder_get_predictions_post_perturb}/{run_name}_90c_combine_predictions_spatial_info.sh"
        final_predictions_output_path = f'{run_folder_FFC}/Predictions_CellProfiler_RF/{run_name}_final_predictions.csv'

        script_head = "\n".join([f"#!/bin/bash",
                                 f"#SBATCH --output {logs_folder}/job_%j_{run_name}_combine_predictions_spatial.log",
                                 f"#SBATCH --error {logs_folder}/job_%j_{run_name}_combine_predictions_spatial.log",
                                 f"#SBATCH --mem={maximum_memory}",                                 
                                 f"#SBATCH --job-name=combine_predictions",
                                 f"#SBATCH --partition={cluster_queue_post_perturb_detection}",
                                 f"#SBATCH --qos={cluster_queue_post_perturb_detection}\n",
                                 f"source /home/${{USER}}/.bashrc\n",
                                 "date; echo\n\n"])
        
        script_body = "\n".join([f'predictions_spatial_path="{spatial_preds_output_path}"',
                                 f'predictions_RF_multiple_substring="{channel_subsets_preds_output_path_basename}"',
                                 f'output_path="{final_predictions_output_path}"',
                                 f'{combine_predictions_script} -s $predictions_spatial_path -p $predictions_RF_multiple_substring -o $output_path\n',
                                 "echo; date",
                                 "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])
        
        with open(script_path, mode='w', encoding='utf-8') as f:
            f.write(script_head)
            f.write(script_body)        
            
        with open(master_script_path, mode='a', encoding='utf-8') as f:
            f.write(f"sbatch --dependency=afterok:$jids {script_path}\n\n")
    print() 

print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")

## 91. Statistical tests - CellProfiler features, post-perturbation, treated vs DMSO

In [ ]:
# group the measurements by plate name; convention: pretreatment is always measurement 1
runs = pd.DataFrame(processed_runs, columns=["run_folder"])
runs["run_name"] = [get_run_name(path) for path in processed_runs]
runs["plate"] = [x.split("__")[0] for x in runs["run_name"]]
runs["measurement"] = [int(x.split("-")[-1].strip("_FFC").split("_")[1]) for x in runs["run_name"]]
runs["cellprofiler_path"] = [f'{runs.loc[i, "run_folder"]}/CellProfiler_measurements/{runs.loc[i, "run_name"]}_merged_cell_measurements.csv' for i in range(len(runs))]
runs["predictions_all_path"] = [f'{runs.loc[i, "run_folder"]}/Predictions_CellProfiler_RF/{runs.loc[i, "run_name"]}_predictions_cellprofiler_rf.csv' for i in range(len(runs))]
runs["predictions_final_path"] = [f'{runs.loc[i, "run_folder"]}/Predictions_CellProfiler_RF/{runs.loc[i, "run_name"]}_final_predictions.csv' for i in range(len(runs))]

plates = sorted(runs.plate.unique())

In [ ]:
master_script_path = f'{scripts_folder}/{analysis_name}_91_test_statistics_cellprofiler_features.sh'
with open(master_script_path, mode='w', encoding='utf-8') as f:
        f.write("#!/bin/bash\n\n")

for plate in tqdm(plates):
    runs_plate = runs[runs["plate"] == plate]
    
    # first measurement
    pretreatment_idx = runs_plate[runs_plate.measurement == 1].index[0]
    # all other treatments
    treat_idxs = runs_plate[runs_plate.index != pretreatment_idx].index
    
    # generate the necessary scripts for all post-treatment measurements
    for idx in treat_idxs:
        run_folder_FFC = runs.loc[idx, "run_folder"]
        run_name = get_run_name(run_folder_FFC)
        
        scripts_folder_test_statistics = f"{scripts_folder}/{run_name}/91_test_statistics_cellprofiler_features"
        os.system(f"mkdir -p {scripts_folder_test_statistics}")
             
        ## get spatial info predictions
        cellprofiler_path = runs.loc[idx, "cellprofiler_path"]
        predictions_final_path = runs.loc[idx, "predictions_final_path"]
        predictions_all_path = runs.loc[idx, "predictions_all_path"]
        test_scores_outfile = f"{hit_calling_folder}/{run_name}_cellprofiler_features_pvals.csv"
        
        script_path = f"{scripts_folder_test_statistics}/{run_name}_91_test_statistics_cellprofiler_features.sh"

        script_head = "\n".join([f"#!/bin/bash",
                                 f"#SBATCH --output {logs_folder}/job_%j_{run_name}_test_statistics_cellprofiler_features.log",
                                 f"#SBATCH --error {logs_folder}/job_%j_{run_name}_test_statistics_cellprofiler_features.log",
                                 f"#SBATCH --mem={maximum_memory}",                                 
                                 f"#SBATCH --job-name=test_stat",
                                 f"#SBATCH --partition={cluster_queue_hit_calling}",
                                 f"#SBATCH --qos={cluster_queue_hit_calling}\n",
                                 f"source /home/${{USER}}/.bashrc\n",
                                 "date; echo\n\n"])
        
        script_body = "\n".join([f'measurement_name="{run_name}"',
                                 f'features_list_path="{features_to_test_path}"',
                                 f'platemap_path="{platemap_path}"',
                                 f'clones_metadata_path="{clones_metadata_path}"',
                                 f'cellprofiler_path="{cellprofiler_path}"',
                                 f'imagelist_path={run_folder_FFC}/Images_metadata/{run_name}_images_list.csv',
                                 f'predictions_final_path="{predictions_final_path}"',
                                 f'predictions_all_path="{predictions_all_path}"',
                                 f'predictions_final_path="{predictions_final_path}"',
                                 f'test_scores_outfile="{test_scores_outfile}"\n',
                                 
                                 f'{test_statistic_cellprofiler_script} -m $measurement_name -f $features_list_path -pm $platemap_path -cm $clones_metadata_path -cp $cellprofiler_path -i $imagelist_path -pf $predictions_final_path -pa $predictions_all_path -o $test_scores_outfile\n',
                                 "echo; date",
                                 "sstat -j $SLURM_JOB_ID.batch --format=JobID,Node,MaxVMSize\n"])
        
        with open(script_path, mode='w', encoding='utf-8') as f:
            f.write(script_head)
            f.write(script_body)
        # master script - prepare dependencies
        with open(master_script_path, mode='a', encoding='utf-8') as f:
            f.write(f"# {run_name}\n")
            f.write(f"jids=$(sbatch --parsable {script_path})\n")
            
print(f"chmod ug+rx {master_script_path}")
print(f"{master_script_path}")